<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Lecture 4*

# Model Validation with Keras - Boston Housing Data

In [1]:
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pandas as pd
import numpy
# fix random seed for reproducibility
numpy.random.seed(45)

c:\users\han-chung lee\.virtualenvs\ds-unit-4-sprint-2-neural-networks-v4qq6gjw\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\han-chung lee\.virtualenvs\ds-unit-4-sprint-2-neural-networks-v4qq6gjw\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\han-chung lee\.virtualenvs\ds-unit-4-sprint-2-neural-networks-v4qq6gjw\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,

In [2]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="ds5-hyperparameter-tuning", entity="ds5")

wandb: ERROR To use wandb on Windows, you need to run the command "wandb run python <your_train_script>.py"


W&B Run: https://app.wandb.ai/ds5/ds5-hyperparameter-tuning/runs/v6fzo0bx

### Load Boston Housing Data

Even though we can import this dataset from Keras, Keras already has it divided up into test and train datasets for us I don't want that for this demonstration, so I'm going to import it from Scikit-Learn instead.

In [3]:
# load dataset
from sklearn.datasets import load_boston
boston_dataset = load_boston()
df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df['MEDV'] = boston_dataset.target
print(df.shape)
df.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Normalizing Input Data

It's not 100% necessary to normalize/scale your input data before feeding it to a neural network, the network can learn the appropriate weights to deal with data of as long as it is numerically represented,  but it is recommended as it can help **make training faster** and **reduces the chances that gradient descent might get stuck in a local optimum**.

<https://stackoverflow.com/questions/4674623/why-do-we-have-to-normalize-the-input-for-an-artificial-neural-network>

In [4]:
from sklearn.preprocessing import StandardScaler

# Split into X and y and turn into numpy arays
y = df.MEDV.values
X = df.drop("MEDV", axis='columns').values

# Scale input data
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)

[[-0.41978194  0.28482986 -1.2879095  ... -1.45900038  0.44105193
  -1.0755623 ]
 [-0.41733926 -0.48772236 -0.59338101 ... -0.30309415  0.44105193
  -0.49243937]
 [-0.41734159 -0.48772236 -0.59338101 ... -0.30309415  0.39642699
  -1.2087274 ]
 ...
 [-0.41344658 -0.48772236  0.11573841 ...  1.17646583  0.44105193
  -0.98304761]
 [-0.40776407 -0.48772236  0.11573841 ...  1.17646583  0.4032249
  -0.86530163]
 [-0.41500016 -0.48772236  0.11573841 ...  1.17646583  0.44105193
  -0.66905833]]


## Model Validation using an automatic verification Dataset

Instead of doing a train test split, Keras has a really nice feature that you can set the validation.split argument when fitting your model and Keras will take that portion of your test data and use it as a validation dataset. 

In [5]:
# Important Hyperparameters
inputs = X.shape[1]
wandb.config.epochs = 50
wandb.config.batch_size = 10

# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
# Fit Model
model.fit(X, y, 
          validation_split=0.33, 
          epochs=wandb.config.epochs,
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
        )

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


wandb: ERROR wandb.init hasn't been called, can't configure run


Train on 339 samples, validate on 167 samples
Epoch 1/50
339/339 [==============================] - 1s 2ms/sample - loss: 607.7502 - mean_squared_error: 607.7501 - val_loss: 225.4418 - val_mean_squared_error: 225.4418
Epoch 2/50
339/339 [==============================] - 0s 858us/sample - loss: 328.0566 - mean_squared_error: 328.0566 - val_loss: 116.6544 - val_mean_squared_error: 116.6544
Epoch 3/50
339/339 [==============================] - 0s 667us/sample - loss: 69.8521 - mean_squared_error: 69.8521 - val_loss: 92.5725 - val_mean_squared_error: 92.5725
Epoch 4/50
339/339 [==============================] - 0s 729us/sample - loss: 27.1007 - mean_squared_error: 27.1007 - val_loss: 80.2582 - val_mean_squared_error: 80.2582
Epoch 5/50
339/339 [==============================] - 0s 608us/sample - loss: 16.4605 - mean_squared_error: 16.4605 - val_loss: 81.2875 - val_mean_squared_error: 81.2875
Epoch 6/50
339/339 [==============================] - 0s 605us/sample - loss: 13.6985 - mean_squar

Epoch 49/50
339/339 [==============================] - 0s 619us/sample - loss: 4.3459 - mean_squared_error: 4.3459 - val_loss: 48.8694 - val_mean_squared_error: 48.8694
Epoch 50/50
339/339 [==============================] - 0s 549us/sample - loss: 4.2753 - mean_squared_error: 4.2753 - val_loss: 49.6407 - val_mean_squared_error: 49.6407


## Model Verification using a Manual Verification Dataset

Even though Keras has this nice feature you can still do train test split and pass it both X_test and y_test datasets as a tuple using the `validation_data` parameter.

In [6]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

# Random Seed
seed = 42
numpy.random.seed(seed)

# Load Dataset
boston_dataset = load_boston()
df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df['MEDV'] = boston_dataset.target

# Split into X and y and turn into numpy arays
y = df.MEDV.values
X = df.drop("MEDV", axis='columns').values

# Scale inputs
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Manual Validation Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

# Important Hyperparameters
inputs = X.shape[1]
epochs = 50
batch_size = 10

# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
# Compile Model
model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])
# Fit Model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)


TypeError: 'NoneType' object is not iterable

## Manual K-fold Cross Validation (regression)

K-fold Cross Validation is the best way to validate your model as it will reduce the variance of your overall accuracy measure by splitting the data into smaller cross validation batches using one of the split out datasets as the validation dataset. The model is then fit K times rotating the validation set until each portion of the data has served as the validation dataset, the accuracy results from each subset are then averaged and reported as the overall test accuracy. The upside is that you can be more confident in your model's final reported accuracy. The downside is that you have to train your model K times which is computationally expensive and just not pheasible for some large networks that can take days to train a single time.

K-fold Cross Validation gets tricky if you want to standardize your data because you will have to standardize your X values for each fold of your model separately. Due to this, we're going to use a Scikit-Learn Pipeline to help us do this for each fold of our dataset. 

Also, it's weird, but cross_val_score returns MSE values as negative values:
<https://github.com/scikit-learn/scikit-learn/issues/2439>

In [ ]:
# Keras imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# sklearn imports
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Random Seed
seed = 42
numpy.random.seed(seed)

# Load Dataset
boston_dataset = load_boston()
df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df['MEDV'] = boston_dataset.target

# Split into X and y and turn into numpy arays
y = df.MEDV.values
X = df.drop("MEDV", axis='columns').values

# Important Hyperparameters
inputs = X.shape[1]
epochs = 50
batch_size = 10

# define base model
def baseline_model():
  # create model
  model = Sequential()
  model.add(Dense(64, input_dim=inputs, activation='relu')) 
  model.add(Dense(64, activation='relu')) 
  model.add(Dense(1))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=epochs, batch_size=batch_size,
    verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(f"K-fold Cross-Val Results - Mean: {-results.mean():.2f} StDev: {results.std():.2f} MSE")

## Manual K-fold Cross Validation (classification)

In [ ]:
# MLP for Pima Indians Dataset with 10-fold cross validation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy

# fix random seed for reproducibility
seed = 42
numpy.random.seed(seed)

# load pima indians dataset
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=8, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # Fit the model
  model.fit(X[train], Y[train], epochs=150, batch_size=10, verbose=0)
  # evaluate the model
  scores = model.evaluate(X[test], Y[test], verbose=0)
  print(f'{model.metrics_names[1]}: {(scores[1]*100):.2f}%') 
  cvscores.append(scores[1]*100)
print(f'{numpy.mean(cvscores):.2f}% +/- {numpy.std(cvscores):.2f}%')

# Hyperparameter Tuning

Hyperparameter tuning is much more important with neural networks than it has been with any other models that we have considered up to this point. Other supervised learning models might have a couple of parameters, but neural networks can have dozens. These can substantially affect the accuracy of our models and although it can be a time consuming process is a necessary step when working with neural networks.

Hyperparameter tuning comes with a challenge. How can we compare models specified with different hyperparameters if our model's final error metric can vary somewhat erratically? How do we avoid just getting unlucky and selecting the wrong hyperparameter? This is a problem that to a certain degree we just have to live with as we test and test again. However, we can minimize it somewhat by pairing our experiments with Cross Validation to reduce the variance of our final accuracy values.

## Hyperparameter Tuning Approaches:

### 1) Babysitting AKA "Grad Student Descent".

If you fiddled with any hyperparameters yesterday, this is basically what you did. This approach is 100% manual and is pretty common among researchers where finding that 1 exact specification that jumps your model to a level of accuracy never seen before is the difference between publishing and not publishing a paper. Of course the professors don't do this themselves, that's grunt work. This is also known as the fiddle with hyperparameters until you run out of time method.

### 2) Grid Search

Grid Search is the Grad Student galaxy brain realization of: why don't I just specify all the experiments I want to run and let the computer try every possible combination of them while I go and grab lunch. This has a specific downside in that if I specify 5 hyperparameters with 5 options each then I've just created 5^5 combinations of hyperparameters to check. Which means that I have to train 3125 different versions of my model Then if I use 5-fold Cross Validation on that then my model has to run 15,525 times. This is the brute-force method of hyperparameter tuning, but it can be very profitable if done wisely. 

When using Grid Search here's what I suggest: don't use it to test combinations of different hyperparameters, only use it to test different specifications of **a single** hyperparameter. It's rare that combinations between different hyperparameters lead to big performance gains. You'll get 90-95% of the way there if you just Grid Search one parameter and take the best result, then retain that best result while you test another, and then retain the best specification from that while you train another. This at least makes the situation much more manageable and leads to pretty good results. 

### 3) Random Search

Do Grid Search for a couple of hours and you'll say to yourself - "There's got to be a better way." Enter Random Search. For Random search you specify a hyperparameter space and it picks specifications from that randomly, tries them out, gives you the best results and says - That's going to have to be good enough, go home and spend time with your family. 

Grid Search treats every parameter as if it was equally important, but this just isn't the case, some are known to move the needle a lot more than others (we'll talk about that in a minute). Random Search allows searching to be specified along the most important parameter and experiments less along the dimensions of less important hyperparameters. The downside of Random search is that it won't find the absolute best hyperparameters, but it is much less costly to perform than Grid Search. 

### 4) Bayesian Methods

One thing that can make more manual methods like babysitting and gridsearch effective is that as the experimenter sees results he can then make updates to his future searches taking into account the results of past specifications. If only we could hyperparameter tune our hyperparameter tuning. Well, we kind of can. Enter Bayesian Optimization. Neural Networks are like an optimization problem within an optimization problem, and Bayesian Optimization is a search strategy that tries to take into account the results of past searches in order to improve future ones. This is the most advanced method but can be a little bit tricky to implement, but there are some early steps with `hyperas` which is Bayesian optimization wrapper for `keras`. 

## What Hyperparameters are there to test?

- batch_size
- training epochs
- optimization algorithms
- learning rate
- momentum
- activation functions
- dropout regularization
- number of neurons in the hidden layer

There are more, but these are the most important.

## Creating a "Test Harness"

You want it to be easy to specify different version of your model and for that model to undergo the same conditions every time with nothing different except for the hyperparameter(s) that you are testing. 

In order to do this we're going to use GridSearchCV from scikit learn to run our model with different hyperparameters and then report back to us the best one. In order to do this we have to wrap our model in either a KerasRegressor or KerasClassifier object to prepare it to work with Scikit-Learn. We also have to put our model specification within a function that we can call in order to generate it. This is required in order to use KerasRegressor or KerasClassifier

## Batch Size

Batch size determines how many observations the model is shown before it calculates loss/error and updates the model weights via gradient descent. You're looking for a sweet spot here where you're showing it enough observations that you have enough information to updates the weights, but not such a large batch size that you don't get a lot of weight update iterations performed in a given epoch. Feed-forward Neural Networks aren't as sensitive to bach_size as other networks, but it is still an important hyperparameter to tune. Smaller batch sizes will also take longer to train. 

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

## Epochs

The number of training epochs has a large and direct affect on the accuracy, However, more epochs is almost always goign to better than less epochs. This means that if you tune this parameter at the beginning and try and maintain the same value all throughout your training, you're going to be waiting a long time for each iteration of GridSearch. I suggest picking a fixed moderat # of epochs all throughout your training and then Grid Searching this parameter at the very end. 

In [ ]:
# define the grid search parameters
param_grid = {'batch_size': [60, 80],
              'epochs': [20, 40, 60]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

## Optimizer

Remember that there's a different optimizers [optimizers](https://keras.io/optimizers/). At some point, take some time to read up on them a little bit. "adam" usually gives the best results. The thing to know about choosing an optimizer is that different optimizers have different hyperparameters like learning rate, momentum, etc. So based on the optimizer you choose you might also have to tune the learning rate and momentum of those optimizers after that. 

## Learning Rate

Remember that the Learning Rate is a hyperparameter that is specific to your gradient-descent based optimizer selection. A learning rate that is too high will cause divergent behavior, but a Learning Rate that is too low will fail to converge, again, you're looking for the sweet spot. I would start out tuning learning rates by orders of magnitude: [.001, .01, .1, .2, .3, .5] etc. I wouldn't go above .5, but you can try it and see what the behavior is like. 

Once you have narrowed it down, make the window even smaller and try it again. If after running the above specification your model reports that .1 is the best optimizer, then you should probably try things like [.05, .08, .1, .12, .15] to try and narrow it down. 

It can also be good to tune the number of epochs in combination with the learning rate since the number of iterations that you allow the learning rate to reach the minimum can determine if you have let it run long enough to converge to the minimum. 

## Momentum

Momentum is a hyperparameter that is more commonly associated with Stochastic Gradient Descent. SGD is a common optimizer because it's what people understand and know, but I doubt it will get you the best results, you can try hyperparameter tuning its attributes and see if you can beat the performance from adam. Momentum is a property that decides the willingness of an optimizer to overshoot the minimum. Imagine a ball rolling down one side of a bowl and then up the opposite side a little bit before settling back to the bottom. The purpose of momentum is to try and escale local minima.

## Activation Functions

We've talked about this a little bit, typically you'l want to use ReLU for hidden layers and either Sigmoid, or Softmax for output layers of binary and multi-class classification implementations respectively, but try other activation functions and see if you can get any better results with sigmoid or tanh or something. There are a lot of activation functions that we haven't really talked about. Maybe you'll get good results with them. Maybe you won't. :) <https://keras.io/activations/>

## Network Weight Initialization

You saw how big of an effect the way that we initialize our network's weights can have on our results. There are **a lot** of what are called initialization modes. I don't understand all of them, but they can have a big affect on your model's initial accuracy. Your model will get further with less epochs if you initialize it with weights that are well suited to the problem you're trying to solve.

`init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']`

## Dropout Regularization and the Weight Constraint

the Dropout Regularization value is a percentage of neurons that you want to be randomly deactivated during training. The weight constraint is a second regularization parameter that works in tandem with dropout regularization. You should tune these two values at the same time. 

Using dropout on visible vs hidden layers might have a different effect. Using dropout on hidden layers might not have any effect while using dropout on hidden layers might have a substantial effect. You don't necessarily need to turn use dropout unless you see that your model has overfitting and generalizability problems.

## Neurons in Hidden Layer 

Remember that when we only had a single perceptron our model was only able to fit to linearly separable data, but as we have added layers and nodes to those layers our network has become a powerhouse of fitting nonlinearity in data. The larger the network and the more nodes generally the stronger the network's capacity to fit nonlinear patterns in data. The more nodes and layers the longer it will take to train a network, and higher the probability of overfitting. The larger your network gets the more you'll need dropout regularization or other regularization techniques to keep it in check. 

Typically depth (more layers) is more important than width (more nodes) for neural networks. This is part of why Deep Learning is so highly touted. Certain deep learning architectures have truly been huge breakthroughs for certain machine learning tasks. 

You might borrow ideas from other network architectures. For example if I was doing image recognition and I wasn't taking cues from state of the art architectures like resnet, alexnet, googlenet, etc. Then I'm probably going to have to do a lot more experimentation on my own before I find something that works.

There are some heuristics, but I am highly skeptical of them. I think you're better off experimenting on your own and forming your own intuition for these kinds of problems. 

- https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/

# Hyperparameter Tuning Tips:

1) Use **at least** 3-fold cross validation. 5-fold or 10-fold would be better if you're patient enough.

2) Sampling your dataset can speed up iterations if you have enough data to justify it.

3) Start with "Coarse Grids" we gave an example of this when we were talking about Learning Rate, start with wide gaps between values that you're testing and then run additional tests with more narrow gaps between values until you find an optimum.

4) Even though we're using Cross Validation, reproducibility can still be a challenge. Don't be afraid to run some tests multiple times to see if you get the same results. Sometimes the accuracy differences are so slight that grid search picking one hyperparameter over another is just luck of the draw.

5) Run your tests in verbose mode. Not only will you be less stressed out wondering if it's actually running, but you will be able to more quickly identify if something has gone haywire by glancing at the output every now and again.

6) Pick a moderate amount of epochs and stick with that all throughout your training, until the very end, then tune your epochs. 

7) As you begin to have a lot of hyperparameters narrowed down, test them along with narrow versions of other hyperparameters in order to catch the small effects of interactions between specific hyperparameters.

8) Being a perfectionist here is going to be a long and painful journey, so please do a cost-benefit analysis of how you're using your time here. Learn the concept, have the experience, but don't spend 3 days hyperparameter tuning your assignment.

## Additional Reading:

I know I sound like a shill for Jason Brownlee, but honestly he has some of the very best articles on the internet in regards to hyperparameter tuning with Keras. I have leaned on his articles heavily when doing hyperparameter tuning before and saw great results from following his advice. He has articles on lots and lots of Keras related topics, so keep an ey out for his website: <http://machinelearningmastery.com> as you look through search results today.

- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
- https://blog.floydhub.com/guide-to-hyperparameters-search-for-deep-learning-models/
- https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/
- https://machinelearningmastery.com/introduction-to-weight-constraints-to-reduce-generalization-error-in-deep-learning/
- https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/